<a href="https://colab.research.google.com/github/movie112/INU-DILAB/blob/main/NLP_pytorch/NLP_ch3_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

파이토치로 배우는 자연어 처리: ch3_2
3. 신경망의 기본 구성 요소

~~3.1 퍼셉트론: 가장 간단한 신경망~~   
~~3.2 활성화 함수~~   
~~3.3 손실 함수~~   
~~3.4 지도 학습 훈련 알아보기~~   
~~3.5 부가적인 훈련 개념~~   
3.6 예제: 레스토랑 리뷰 감성 분류하기

## 3.6 예제: 레스토랑 리뷰 감성 분류하기

#### 3.6.1 옐프 리뷰 데이터셋
<https://github.com/rickiepark/nlp-with-pytorch/blob/main/chapter_3/3_5_Classifying_Yelp_Review_Sentiment.ipynb>

#### 데이터 벡터화 클래스

##### dataset class
- Dataset 클래스는 추상화된 반복자(iterater)
- 새로운 데이터셋을 사용할 때는 먼저 Dataset 클래스를 상속하여 `__getitem__()`와 `__len__()`메서드를 구현
  - 이 예에서는 Dataset class를 상속하여 ReviewDataset 클래스를 만들고 `__getitem__()`와 `__len__()`
  - ReviewDataset 클래스 안에서 예제 데이터셋과 함께 다양한 파이토치 유틸리티 사용(ex. DataLoader)
  - 이 클래스는 ReviewVectorizer 클래스에 크게 의존
  - 우선 `ReviewVectorizer`는 리뷰 텍스트를 수치 벡터로 변환하는 클래스
  - 벡터화 간계를 거쳐야 신경망이 텍스트 데이터를 다룰 수 있다. 
  - 전체적인 셀계는 게이터 포인트 하나에 벡터화 로직을 적용하는 데이터셋 클래스를 구현하는 것
  - 그다음 DataLoader 클래스가 데이터셋에서 샘플링하고 모아서 미니배치 만듦

---

- 이 책의 예제 대부분 Vocabulary, Vectorizer, DataLoader 클래스를 사용해 중요한 파이프라인을 수행
- 텍스트 입력을 벡터의 미니배치로 바꿉니다. 이 파이프라인은 전처리된 텍스트를 사용합니다. 즉, 각 데이터 포인트를 토큰의 집합입니다.
- 이 예에서 토큰은 단어에 해당합니다. 이어지는 절에 등장하는 세 클래스는 각 토큰을 정수에 매핑하고, 이 매핑을 각 데이터 포인트에 적용해 벡터 형태로 변환합니다. 그다음 벡터로 변환한 데이터 포인트를 모델을 위해 미니배치오 모읍니다. 
---

##### Vocabulary
- 텍스트를 벡터의 미니배치로 바꾸는 첫 번째 단계: 토큰은 정수로 매핑하기
- 두 딕셔너리를 Vocabulary 클래스에 캡슐화합니다. Vovabulary 클래스는 딕셔너리에 사용자가 코튼을 추가하면 자동으로 인덱스를 증가시키고 특별토큰도 관리한다.
  - UNK(unknown): 테스트할 째 훈련에서 본 적이 없는 토큰은 처리할 수 있다.
  - Vectorizer 클래스에서 자주 등장하지 않은 토큰을 제한할 것이다. 이런 토큰이 훈련 과정에서 UNK 토큰으로 나타남
  - 이는 Vocabulary 클래스가 사용하는 메모리를 제한하는 데 필수
- Vocabulary에 새 토큰을 추가하기 위해 `add_token()`을 호출
- 토큰에 해당하는 인덱스를 추출할 때는 `lookup_token()` 호출
- 특정 인덱스에 해당하는 토큰을 추출할 때는 `lookup_index()` 호출

---

##### Vectorizer
- 텍스트 데이터를 벡터의 미니배치로 바꾸는 두 번째 단계: 입력 데이터 포인트의 토큰을 순회하면서 각 토큰을 정수로 바꾸기
- 이 반복 과정의 결과는 벡터, 이 벡터가 다른 데이터 포인트에서 만든 벡터와 합쳐지므로 Vetorizer 클래스는 리뷰의 단어를 정수로 매핑한 Vocabulary를 캡슐화
---
- `from_dataframe()`: 판다그 데이터프레임을 순화하면서 두 가지 작업 수행
  - 데이터셋에 있는 모든 토큰의 빈도수를 카운트
  - 키워드 매개변수 cutoff에 지정한 수보다 빈도가 높은 토큰만 사용하는 Vocabulary 객체를 만듦
    - 실제로 이 메서드는 최소한 cutoff 횟수보다 많이 등장하는 단어를 모두 찾아 Vocabulary 객체에 추가
    - UNK 토큰도 추가하므로 `lookuo_token()`을 호출할 때 포함되지 않은 단어는 모두 `unk_index` 반환
- `vectorize()`:  Vectorizer 클래스의 핵심기능을 캡슐화, 매게변수로 리뷰 문자열을 받고 벡터 표현을 반환


---
##### DataLoader
- 텍스트를 벡터로 변환하는 미니배치 파이프라인의 마지막 단계는 백터로 변환한 데이터 포인트 모으기
- 파이토치 내장 클래스인 DataLoader는 신경망 훈련에 필수인 미니배치로 모으는 작업을 편하게 해줍니다.
  - Dataset(ReviewDataset), batch_size 및 몇 가지 편리한 키워드를 매개변수로 받는다.
  - 이 클래스의 객체는 Dataset 클래스가 제공한 데이터 포인트를 순회하는 iterator
---

- `generate_betches()` 로 DataLoader을 감쌌다, CPU, GPU 간 데이터를 간편하게 전환하는 getnerater
- 파이토치의 Dataset 클래스를 상속하려면 `__getitem__()`, `__len()__` 메서드를 구현해햐 한다는 점을 기억하세요. 이를 통해 DataLoader 클래스가 인덱스를 사용해 데이터셋을 순회할 수 있습니다.


---

#### 3.6.4 퍼셉트론 분류기
- perceptron 분류기를 다시 구현한 모델을 사용합니다.
- `ReviewClassifie`r 클래스는 파이토치의 Module 클래스를 상속하고 단일 출력을 만드는 Linear 층을 하나 생성합니다.
- 이진 분류 문제이므로 출력이 하나입니다.
- 비선형 활성화 함수로는 시그모이드 사용

---

- `forward()`:  선택적으로 sigmoid를 적용하는 매개변수가 존재
  - 이진 분류 문제에서 이진 cross-entropy 손실(nn.BCDLoss())이 가장 적절한 손실함수
  - 시그모이드와 손실 함수를 함께 사용할 때는 수치 안정성 이슈가 있습니다. 파이토치는 sigmoid 없이 간편하게 사용할 수 있고 수치적으로 안정된 계산을 위한 `BCEWithLogistLoss()` 제공

---

#### 3.6.5 모델 훈련
- 모델 훈련의 구성요소 & 구성 요소를 모델 및 데이터셋과 연결하여 모델 파라미터를 조정하고 성능을 높이는 방법
- 훈련과정의 핵심은 모델을 만들고, 데이터셋을 순회하고, 입력 데이터에서 모델의 출력을 계산하고, 손실을 계산하고, 손실에 비례하여 모델을 수정하는 것
- 고수준은 결정을 조정하기 편하도록 모든 결정 요소를 관리하는 `args` 객체를 사용
  - 내장 argparse 패키지의 Namespace 클래스를 사용 - 속성 딕셔너리를 잘 캡슐화하고 정적 분석기(static anlayzer)와 함계 잘 동작, 또한 명령줄 기반 훈련 방식을 구축한다면 다른 코드를 바꾸지 않고 argparse의 Argument Parser로 바꿀 수 있다.
- `train_state`: 훈련 과정 정보를 담은 작은 딕셔너리
  - 모델을 훈련하면서 추적할 정보가 늘어날수록 이 딕셔너리 커짐
- 훈련 상태 딕셔너리를 설명한 후 모델 훈련을 실행하기 위해 생성할 일련의 객체를 소개합니다,
  - 여기에는 모델 자체와 데이터셋, 옵티마이저, 손실한수가 포함됩니다.
- 마지막으로 훈련 반복으로 이 절을 마무리하고 기본적인 파이토치 최적화 과정을 시연합니다.

---




##### 훈련 준비
- [코드 3-20] 예에서 만들 훈련 구성 요소를 보임
- 첫 번째 요소: 훈련 상태를 초기화하는 함수
  - 훈련 상태가 복잡한 정보를 다룰 수 있도록 `args` 객체를 매개변수로 받습니다.
  - 에포크 인덱스, 훈련 손실 리스트, 훈련 정확도, 검증 손실, 검증 정확도, 테스트 손실과 테스트 정확도 포함
- 다음으로는 데이터셋과 모델을 망듭니다. 
  - 데이터셋 클래스에서 Vectorizer를 만듭니다.
  - if 문 안에서 데이터셋을 만든다.
  - 이전에 만든 Vectorizer 객체를 로드할지 새로운 객체를 만들어 디스크에 저장할지 결정
  - GPU 장치를 사용할 수 있는지 확인하여 사용자가 (args.cuda에) 지정한 장치로 모델을 이동
  - 데이터와 모델이 같은 장치에 있어야 하므로 훈련 반복 안에서 호출되는 `generate_batches()` 함수에서 이 장치 사용
- 손실 함수와 옵티마이저가 초기화 단계의 마지막 요소
  - Adam은 다른 옵티마이저들보다 성능이 뛰어납니다.


##### 훈련 반복
- 훈련 반복은 초기화한 객체를 사용해 모델의 성능이 높아지도록 모델 파라미터를 업데이트
- 구체적으로 두 개의 반복문으로 구성
  - 내부: 데이터셋의 미니배치에 대해 반복 수행
  - 외부: 내부 반복문을 여러 번 반복
  - 애부에서 미이배치마다 손실을 계산하고 옵티마이저가 모델 파라미터를 업데이트합니다.

--- 

- 첫 번째 라인에서 epoch 횟수만큼 for 반복문 실행
  - epoch 횟수는 하이퍼파라미터로 바꿀 수 있으며 주어진 데이터셋에 훈련을 몇번이나 반복할지 결정
  - 실전에서는 조기 종료 조건 등을 이요해 끝에 도달하기 전에 반복문을 종료
  - 반복문 상단에서 몇 가지를 정의하고 초기화
    - 훈련 상태의 epoch index를 설정
    - 분할한 데이터셋 중 어떤 세트를 사용할지 지정(처음에는 'train' -> 에포크 끝에서 모델 성능을 측정할 때 'val'지정 -> 모델의 최종 성능을 측정할 때 'test'
    - 데이터 셋을 구성한 방식 때문에 항상 `generate_batches()`를 호출하기 전에 이런 세트를 설정해야 함
    - `batch_generator`를 만든 후 두 개의 실수 변수를 준비해 배치 간의 손실과 정확도를 기록
    - 분류기의 `train()` 매서드를 호훌해 모델이 훈련 모드에 있고 모델 파라미터를 수정할 수 있다고 지정
    - 이렇게 하면 드롭아웃 같은 규제 사용가능

---

- 훈련 반복문 다음 부분은 `batch_gemerator`에서 추출한 훈련 배치를 순회하면서 모델 파라미터를 업데이트하는 핵십 연산 수행
  - 배치 반복마다 `optimizer.zero_grad()`로 optimizer의 gradient 초기화
  - 모델의 출력을 계산
  - 손실 함수를 사용해 모델 출력과 타깃(정답 클래스 레이블) 사이의 손실 계산
  - 손실 객체의 `backward()`로 각 파라미터에 gradient 전파
  - optimizer는 전파한 gradient를 사용해 `optimizer.step()`로 파라미터 업데이트 수행

---

- 훈련 세트의 배치로 내부 반복문 수행이 끝아면 부가 정보 기록과 초기화 연산을 수행
  -  훈련 상테를 최종 손실과 정확도 값으로 업데이트
  - 새로운 배치 제너레이터, 이동평균 손실, 이동 평균 정확도를 만든다.
  - 검증 세트에 사용하는 반복문은 훈련 셋과 비슷하므로 같은 변수 재사용
  - 분류기 `train()`과 반대 작업을 하는 `eval()` 메서드를 호출한다는 차이
  - `eval()`: 모델 파라미터를 수정 못하게 하고 드롭아웃 비활성화
    - 손실 계산하지 않고 gradient를 파라미터로 전파하지 않음
    - 대신 데이터를 모델의 성능을 측정하는 데 사용
    - 훈련셋의 성능과 검증셋의 성능의 차이가 크면 모델이 훈련 셋에 과대적합되었을 가능성이 높아 모델이나 훈련 과정 수정 필요
- 검증 셋을 순화하면서 게산한 검증 손실과 정확도를 저자하고 나면 외부 for 반복문이 끝남


---

#### 3.6.6 평가, 추론, 분석
- 모델을 훈련하고 난 다음 따로 떼어 놓은 데이터에서 얼마나 잘 동작하는지 평가하고 새로운 데이터에 대한 추론을 만들 수 있다.
- 모델 가중치를 분석해 무엇을 학습했는지 알아볼 수 있다.

##### 테스트 데이터로 평가하기
- 코드는 훈련 코드의 검증 반복문과 거의 같다.
  - 사용할 데이터를 `val` 대신 `test`로 지정한다는 차이
  - 두 데이터 셋 간 차이는 케스트 셋을 가능한 한 적게 사용한다는 점
- 테스트 셋에서 훈련한 모델을 실행하여 새로운 모델 구조(ex.층의 크기)를 결정하고 새로 훈련한 모델을 테스트 세트에서 다시 평가할 때마다 점점 더 테스트 셋에 편향된 모델을 얻게 됨
  - 즉, 이 과정을 너무 반복하면 새 데이터에서 측정하려고 데이터 셋을 따로 떼어 놓은 의미가 없어짐

##### 새 데이터 포인트 추론하여 분류
- 모델을 평가하는 또 다른 방법은 새로운 데이터에서 추론한 후 모델이 잘 동작하는지 정성적인 평가를 내리는 것 

##### 모델 가중치 분석
- 마지막 방법은 가중치를 분석해 올바른 값인지 평가하는 것